In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 1424, done.
remote: Counting objects: 100% (273/273), done.
remote: Compressing objects: 100% (131/131), done.
remote: Total 1424 (delta 201), reused 189 (delta 139), pack-reused 1151
Receiving objects: 100% (1424/1424), 78.94 MiB | 23.40 MiB/s, done.
Resolving deltas: 100% (1031/1031), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 24.0 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In functio

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
from optuna.distributions import FloatDistribution, IntDistribution, CategoricalDistribution
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv
/kaggle/input/training-tuned-models/__results__.html
/kaggle/input/training-tuned-models/__notebook__.ipynb
/kaggle/input/training-tuned-models/ials_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/ials_train_validation.zip
/kaggle/input/training-tuned-models/__output__.json
/kaggle/input/training-tuned-models/ials_all.zip
/kaggle/input/training-tuned-models/custom.css
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/run_all_algorithms.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/cython_example_FunkSVD.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/LICENSE
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/.gitignore
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/rp3-tuning.ipynb
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/README.md
/kaggle/input/trai

# Configurations

In [5]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [6]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [7]:
from os import system, getpid, walk
from psutil import Process
from colorama import Fore, Style, init
from IPython.display import display, HTML

def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    print(style + color + text + Style.RESET_ALL)
    
def GetMemUsage():   
    pid = getpid()
    py = Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return f"RAM memory GB usage = {memory_use :.4}"

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 0.2158


In [8]:
config = {
    'train_ials': False,
    'train_ease_r': False,
    'train_nmf': False,
    'train_slim_bpr': False,
    'train_slim_elastic': True,
    'save_github': True,
    'copy_ials_models': True
}

# Data Import

In [9]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [10]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [11]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [12]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender, SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [13]:
URM_train_complete, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_complete, train_percentage = 0.8)

# Training best model

In [14]:
if config['train_ials']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_ials_v2.json', 'r') as params_file:
        params = json.load(params_file)
        
    recommender = IALSRecommender(URM_train)
    recommender.fit(**params)
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='ials_xgboost_train_validation')
    
    # too big to be saved in this way (need to be loaded from this notebook)
#     if config['save_github']: 
#         upload_file(
#             '/kaggle/working/ials_xgboost_train_validation.zip',
#             'models/ials_xgboost_train_validation.zip', 
#             'IALS best model on the xgboost training set for tuning'
#         )
        
    recommender = IALSRecommender(URM_train_complete)
    recommender.fit(**params)
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='ials_train_validation')
    
#     if config['save_github']: 
#         upload_file(
#             '/kaggle/working/ials_train_validation.zip',
#             'models/ials_train_validation.zip', 
#             'IALS best model on the train-validation set'
#         )
    
    recommender = IALSRecommender(URM_all)
    recommender.fit(**params)
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='ials_all')
    
#     if config['save_github']: 
#         upload_file(
#             '/kaggle/working/ials_all.zip', 
#             'models/ials_all.zip', 
#             'IALS best model on all the data'
#         )

elif config['copy_ials_models']:
    shutil.copyfile(
        '/kaggle/input/training-tuned-models/ials_xgboost_train_validation.zip', 
        '/kaggle/working/ials_xgboost_train_validation.zip', 
    )
    shutil.copyfile(
        '/kaggle/input/training-tuned-models/ials_train_validation.zip', 
        '/kaggle/working/ials_train_validation.zip', 
    )
    shutil.copyfile(
        '/kaggle/input/training-tuned-models/ials_all.zip', 
        '/kaggle/working/ials_all.zip', 
    )

In [15]:
if config['train_ease_r']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_EASE_R.json', 'r') as params_file:
        params_EASE_R = json.load(params_file)
        
    recommender = EASE_R_Recommender(URM_train)
    recommender.fit(topK=params_EASE_R['topK'], l2_norm=params_EASE_R['l2_norm'])
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='ease_r_xgboost_train_validation')
    
    if config['save_github']: 
        upload_file(
            '/kaggle/working/ease_r_xgboost_train_validation.zip',
            'models/ease_r_xgboost_train_validation.zip', 
            'ease r best model on the xgboost training set for tuning'
        )
        
    recommender = EASE_R_Recommender(URM_train_complete)
    recommender.fit(topK=params_EASE_R['topK'], l2_norm=params_EASE_R['l2_norm'])
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='ease_r_train_validation')
    
    if config['save_github']: 
        upload_file(
            '/kaggle/working/ease_r_train_validation.zip',
            'models/ease_r_train_validation.zip', 
            'ease r best model on the train-validation set'
        )
    
    recommender = EASE_R_Recommender(URM_all)
    recommender.fit(topK=params_EASE_R['topK'], l2_norm=params_EASE_R['l2_norm'])
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='ease_r_all')
    
    if config['save_github']: 
        upload_file(
            '/kaggle/working/ease_r_all.zip', 
            'models/ease_r_all.zip', 
            'ease r best model on all the data'
        )

In [16]:
if config['train_nmf']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_NMF.json', 'r') as params_file:
        params_NMF = json.load(params_file)
        
    recommender = NMFRecommender(URM_train)
    recommender.fit(
        init_type="nndsvda", l1_ratio=params_NMF['l1_ratio'], beta_loss="frobenius",
        num_factors=params_NMF["num_factors"], solver=params_NMF["solver"]
    )
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='nmf_xgboost_train_validation')
    
    if config['save_github']: 
        upload_file(
            '/kaggle/working/nmf_xgboost_train_validation.zip',
            'models/nmf_xgboost_train_validation.zip', 
            'nmf best model on the xgboost training set for tuning'
        )
        
    recommender = NMFRecommender(URM_train_complete)
    recommender.fit(
        init_type="nndsvda", l1_ratio=params_NMF['l1_ratio'], beta_loss="frobenius",
        num_factors=params_NMF["num_factors"], solver=params_NMF["solver"]
    )
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='nmf_train_validation')
    
    if config['save_github']: 
        upload_file(
            '/kaggle/working/nmf_train_validation.zip',
            'models/nmf_train_validation.zip', 
            'nmf best model on the train-validation set'
        )
    
    recommender = NMFRecommender(URM_all)
    recommender.fit(
        init_type="nndsvda", l1_ratio=params_NMF['l1_ratio'], beta_loss="frobenius",
        num_factors=params_NMF["num_factors"], solver=params_NMF["solver"]
    )
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='nmf_all')
    
    if config['save_github']: 
        upload_file(
            '/kaggle/working/nmf_all.zip', 
            'models/nmf_all.zip', 
            'nmf best model on all the data'
        )

In [17]:
if config['train_slim_elastic']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_slim.json', 'r') as params_file:
        params_slim_elastic = json.load(params_file)
        
    recommender = SLIMElasticNetRecommender(URM_train)
    recommender.fit(**params_slim_elastic)
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='slim_elastic_xgboost_train_validation')
    
    if config['save_github']:
        upload_file(
            '/kaggle/working/slim_elastic_xgboost_train_validation.zip',
            'models/slim_elastic_xgboost_train_validation.zip', 
            'slim elastic best model on the xgboost training set for tuning'
        )
        
    recommender = SLIMElasticNetRecommender(URM_train_complete)
    recommender.fit(**params_slim_elastic)
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='slim_elastic_train_validation')
    
    if config['save_github']: 
        upload_file(
            '/kaggle/working/slim_elastic_train_validation.zip',
            'models/slim_elastic_train_validation.zip', 
            'slim elastic best model on the train-validation set'
        )
    
    recommender = SLIMElasticNetRecommender(URM_all)
    recommender.fit(**params_slim_elastic)
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='slim_elastic_all')
    
    if config['save_github']:
        upload_file(
            '/kaggle/working/slim_elastic_all.zip', 
            'models/slim_elastic_all.zip', 
            'slim elastic best model on all the data'
        )

SLIMElasticNetRecommender: URM Detected 851 ( 6.5%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 476 ( 2.1%) items with no interactions.
SLIMElasticNetRecommender: Processed 9749 (43.6%) in 5.00 min. Items per second: 32.49
SLIMElasticNetRecommender: Processed 18951 (84.8%) in 10.00 min. Items per second: 31.58
SLIMElasticNetRecommender: Processed 22347 (100.0%) in 12.14 min. Items per second: 30.68
SLIMElasticNetRecommender: Saving model in file '/kaggle/working/slim_elastic_xgboost_train_validation'
SLIMElasticNetRecommender: Saving complete
SLIMElasticNetRecommender: URM Detected 627 ( 4.8%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 247 ( 1.1%) items with no interactions.
SLIMElasticNetRecommender: Processed 8636 (38.6%) in 5.00 min. Items per second: 28.78
SLIMElasticNetRecommender: Processed 17576 (78.7%) in 10.00 min. Items per second: 29.29
SLIMElasticNetRecommender: Processed 22347 (100.0%) in 12.91 min. Items per second: 28.85
S

In [18]:
if config['train_slim_bpr']:
    with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_Slim_BPR.json', 'r') as params_file:
        params_Slim_BPR = json.load(params_file)
        
    recommender = SLIM_BPR_Python(URM_train)
    recommender.fit(**params_Slim_BPR)
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='slim_bpr_xgboost_train_validation')
    
    if config['save_github']:
        upload_file(
            '/kaggle/working/slim_bpr_xgboost_train_validation.zip',
            'models/slim_bpr_xgboost_train_validation.zip', 
            'slim bpr best model on the xgboost training set for tuning'
        )
        
    recommender = SLIM_BPR_Python(URM_train_complete)
    recommender.fit(**params_Slim_BPR)
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='slim_bpr_train_validation')
    
    if config['save_github']: 
        upload_file(
            '/kaggle/working/slim_bpr_train_validation.zip',
            'models/slim_bpr_train_validation.zip', 
            'slim bpr best model on the train-validation set'
        )
    
    recommender = SLIM_BPR_Python(URM_all)
    recommender.fit(**params_Slim_BPR)
    
    recommender.save_model(folder_path='/kaggle/working/', file_name='slim_bpr_all')
    
    if config['save_github']:
        upload_file(
            '/kaggle/working/slim_bpr_all.zip', 
            'models/slim_bpr_all.zip', 
            'slim bpr best model on all the data'
        )